# Note

This notebook produces an error we could not solve: a [related issue](https://www.kaggle.com/general/141350) has been posted on kaggle.

In [1]:
#run only once per kaggle session
!pip install /kaggle/input/bert-for-tf2/py-params-0.8.2/py-params-0.8.2/
!pip install /kaggle/input/bert-for-tf2/params-flow-0.7.4/params-flow-0.7.4/
!pip install /kaggle/input/bert-for-tf2/bert-for-tf2-0.13.2/bert-for-tf2-0.13.2/
!pip install sentencepiece

Processing /kaggle/input/bert-for-tf2/py-params-0.8.2/py-params-0.8.2
  Created wheel for py-params: filename=py_params-0.8.2-py3-none-any.whl size=4632 sha256=407e31cf8ea84a13519da41a3f6c24f1b0fe7418ea998c813e02543241630ec6
  Stored in directory: /root/.cache/pip/wheels/51/4d/e7/66d3d48420c6b97ada4739885ff670f5fe0e7f062e7c837e50
Successfully built py-params
Processing /kaggle/input/bert-for-tf2/params-flow-0.7.4/params-flow-0.7.4
  Created wheel for params-flow: filename=params_flow-0.7.4-py3-none-any.whl size=16194 sha256=d6bab5c8b5fd50a67c0e866dc6f6ad84e6733487e979f9a9fefd8b38480b5325
  Stored in directory: /root/.cache/pip/wheels/43/47/aa/df889d7c3cc5194153ae36670542eb434b4ef5cdbe8969e3ab
Successfully built params-flow
Processing /kaggle/input/bert-for-tf2/bert-for-tf2-0.13.2/bert-for-tf2-0.13.2
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.13.2-py3-none-any.whl size=50796 sha256=72d5c395980e8ff3233495e522c822c464a6d6d1ae9e9cd4a4994b78b2a992f2
  Stored in directory: /ro

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from bert.tokenization.bert_tokenization import FullTokenizer
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import pickle
import numpy as np
import tensorflow.keras
from keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0


Using TensorFlow backend.


Load TPU

In [3]:
tf.compat.v1.disable_eager_execution()
assert not tf.executing_eagerly()
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
print("REPLICAS: ", tpu_strategy.num_replicas_in_sync)

REPLICAS:  8


In [4]:
# load data

with open('../input/imbddata/train_input.pickle', 'rb') as f:
     train_input= pickle.load(f)

    
print("done")


done


In [5]:
%%time


max_seq_length = 512  
max_len = max_seq_length
dropouts = [0.3, 0.4,0.5] #0.6 già fatto


for drop in dropouts:
    with tpu_strategy.scope():
        with open('../input/imbddata/train_input.pickle', 'rb') as f:
             train_input= pickle.load(f)
        train_labels = np.load("../input/imbddata/train_labels.npy")
        METRICS = [ tensorflow.keras.metrics.BinaryAccuracy(name='accuracy'), tensorflow.keras.metrics.AUC(name='auc')]
        
        input_word_ids = tensorflow.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
        input_mask = tensorflow.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
        segment_ids = tensorflow.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
        bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/./bert_en_uncased_L-12_H-768_A-12/1",trainable=True)
        _ , sequence_output = bert_layer([input_word_ids, input_mask, segment_ids]) 
        clf_output = sequence_output[:, 0, :] 
        mean = tf.reduce_mean(sequence_output, 1) #testare due opzioni
        out = tensorflow.keras.layers.Dense(512, activation='relu')(mean)
        out = tf.nn.dropout(out,drop)
        final = tensorflow.keras.layers.Dense(1, activation='sigmoid')(out)
        model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=final)
        model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=METRICS)
    model.summary()
    train_labels = train_labels.astype(float)
    history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=6,
    batch_size=128* tpu_strategy.num_replicas_in_sync, #12
    verbose = 100)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

InvalidArgumentError: From /job:worker/replica:0/task:0:
Unsuccessful TensorSliceReader constructor: Failed to get matching files on /tmp/tfhub_modules/041d84cb334a865509a787b4c9177fb6c7bba8c2/variables/variables: Unimplemented: File system scheme '[local]' not implemented (file: '/tmp/tfhub_modules/041d84cb334a865509a787b4c9177fb6c7bba8c2/variables/variables')
	 [[node RestoreV2 (defined at /opt/conda/lib/python3.6/site-packages/tensorflow_hub/module_v2.py:95) ]]

Original stack trace for 'RestoreV2':
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/